In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd

# Option Valuation Methods - Homework 2
### Andrei Stefan


## Exercise 1: Implied volatility

#### 1a)